<a href="https://colab.research.google.com/github/gatherheart/Classification_With_BERT/blob/main/news_classifications_pytorch_kobert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [148]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3
!pip install torch

     |████████████████████████████████| 757kB 10.1MB/s 
     |████████████████████████████████| 3.0MB 22.8MB/s 
  Found existing installation: tokenizers 0.9.4
    Uninstalling tokenizers-0.9.4:
      Successfully uninstalled tokenizers-0.9.4
  Found existing installation: transformers 4.0.1
    Uninstalling transformers-4.0.1:
      Successfully uninstalled transformers-4.0.1


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-jesns6hw
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-jesns6hw
  Created wheel for kobert: filename=kobert-0.1.2-cp36-none-any.whl size=12734 sha256=3a709a570db29c247bb4b8b2b36b6412ef407db015d26b24359256ad232052c7
  Stored in directory: /tmp/pip-ephem-wheel-cache-dvw162ae/wheels/a2/b0/41/435ee4e918f91918be41529283c5ff86cd010f02e7525aecf3
Successfully built kobert


In [149]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
from tqdm import tqdm, tqdm_notebook, notebook

In [150]:
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [151]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [152]:
import tensorflow as tf

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import random
import time
import datetime

In [155]:
##GPU 사용 시
device = torch.device("cuda:0")

In [156]:
bertmodel, vocab = get_pytorch_kobert_model()

using cached model
using cached model


In [157]:
BASE_PATH = '/content/drive/MyDrive/hyun_data'

In [158]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Sat Dec 12 04:25:16 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.45.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P0    33W / 250W |   8037MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [159]:
df = pd.read_csv(BASE_PATH+'/preprocessed_norm.csv', index_col=[0])

In [160]:
labels_csv = pd.read_csv(BASE_PATH+'/labels_dict2.csv', index_col=[0])

In [161]:
labels_csv

,문화>방송_연예,국제>중국,사회>사회일반,정치>국회_정당,사회>교육_시험,IT_과학>모바일,정치>행정_자치,사회>사건_사고,문화>문화일반,정치>청와대,사회>여성,국제>유럽_EU,경제>자원,국제>미국_북미,사회>날씨,사회>의료_건강,IT_과학>콘텐츠,경제>증권_증시,정치>선거,국제>일본,사회>미디어,문화>음악,국제>중남미,국제>중동_아프리카,경제>유통,문화>출판,경제>자동차,사회>환경,경제>금융_재테크,사회>장애인,문화>전시_공연,경제>부동산,IT_과학>IT_과학일반,문화>요리_여행,IT_과학>과학,경제>서비스_쇼핑,경제>외환,미분류,문화>미술_건축,문화>생활,경제>경제일반,국제>아시아,정치>정치일반,경제>국제경제,경제>취업_창업,IT_과학>보안,정치>외교,경제>무역,문화>영화,정치>북한,경제>산업_기업,문화>종교,경제>반도체,문화>학술_문화재,사회>노동_복지,국제>국제일반,IT_과학>인터넷_SNS,국제>러시아
0,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57


In [162]:
df.columns

Index(['date', 'title', 'topic', 'keyword', 'input', 'body', 'label'], dtype='object')

In [163]:
df[df.label==0]

,date,title,topic,keyword,input,body,label
8,20201128,"'온앤오프' 이지아, '데뷔 후 최초' 베일 싸여있던 일상 공개",문화>방송_연예,"온앤오프,이지아,데뷔,베일,일상,공개,28일,방송,tvN,온앤오프,배우,이지아,반전...",이지아 온앤오프 덕후 한경닷컴 김나경 tvn 은하계 ufo 동작 하나하나 외계인설 ...,28일(토) 방송하는 tvN ’온앤오프‘에서는 배우 이지아의 반전 매력 넘치는 일상...,0
10,20201128,"'런닝맨' 김소연, '팬심' 폭발시킨 멤버 정체는?",문화>방송_연예,"런닝맨,김소연,팬심,폭발,멤버,정체,배우,김소연,팬심,고백,29일,방송,SBS,런닝...",김소연 런닝맨 팬심 이상우 멤버들 리액션 반박해 180도 다들 거짓말 펜트하우스 1...,배우 김소연이 수줍은 팬심을 고백한다.\n오는 29일 오후 방송되는 SBS ‘런닝맨...,0
15,20201128,"‘트로트의 민족’ 김소연, 우승 후보→패자부활전 합격할까[M+TV인사이드]",문화>방송_연예,"트로트,민족,김소연,우승,후보,패자부활전,합격,TV인사이드,트로트,민족,유력,우승,...",트로트 박하명 긴장감 김소연 안성준 시청률 신명근 생존자 패자부활전 장명서 나미애 ...,‘트로트의 민족’ 유력 우승 후보 김소연이 패자부활전에 나선 가운데 그의 생존에 관...,0
21,20201128,"'아는 형님' 조병규, 즉석 액션부터 성대모사까지 매력 대방출!",문화>방송_연예,"형님,조병규,즉석,액션,성대모사,매력,대방출,배우,조병규,예능,블루칩,등극,방송,J...",조병규 김세정 눈물 성대모사 유준상 친밀도 만큼 촬영장 웃픈 블루칩 자유자재 전학생...,배우 조병규가 넘치는 끼로 예능 블루칩에 등극했다.\n \n28일 방송되는 JTBC...,0
67,20201128,‘온앤오프’ 이지아 “UFO 출몰 지역을 간 적도 있다”,문화>방송_연예,"온앤오프,이지아,UFO,출몰,지역,28일,방송,tvN,온앤오프,배우,이지아,반전,매...",이지아 온앤오프 덕후 ufo tvn 동작 하나하나 은하계 외계인설 싱어송라이터 놀라...,[헤럴드경제 = 서병기 선임기자]28일(토) 밤 방송되는 tvN ’온앤오프‘에서는 ...,0
...,...,...,...,...,...,...,...
19069,20201201,영화보며 시간 여행 자연 힐링 독서하며 언택트 송년,문화>방송_연예,"영화,여행,자연,힐링,독서,언택트,송년,연말,영화,리틀,포레스트,Little,개봉,...",구멍가게 일본 순천만 미국 곽재구 루나나 해남 목포 원스 혜원 보이저 시간여행 드루...,[연말에 보면 좋은 영화]\n▲리틀 포레스트(Little Forest /감독 임순례...,0
19304,20201201,"[종합] 아이랜드 통해 데뷔한 '엔하이픈' ""7명 전원이 센터급 비주얼"" 연말 신...",문화>방송_연예,"아이랜드,데뷔,엔하이픈,전원,센터,비주얼,연말,신인상,사진,빌리프랩,제공,빅히트엔터...",엔하이픈 아이랜드 빅히트 니키 희승 제이크 제이 타이틀곡 성훈 7명 신인상 팬들 방...,[사진= 빌리프랩 제공] \n \n \n 빅히트엔터테인먼트와 Mnet의 프로젝트 ...,0
19581,20201130,"[이웃사랑] 뇌병변에 유방암까지 온 아내, 홀로 고군분투 중인 남편",문화>방송_연예,"뇌병변,유방암,아내,고군분투,남편,결혼식,뇌병변,장애,남편,4년,병간호,유방암,생활...",주희 영석 결혼식 예주 유방암 중국집 대구은행 병간호 핸드폰 일자리 생활비 매일신문...,지난 27일 달서구 감삼동의 한 주택 2층. 쿵쾅쿵쾅 계단 위로 한 여자아이가 급히...,0
19598,20201130,전효성 다이어트 “3개월만에 허리 26인치→23.5인치”,문화>방송_연예,"3개월,전효,다이어트,허리,인치,23.5인치,가수,전효,다이어트,영상,공개,화제,전...",전효성 전효 3개월 다이어터 필라테스 배달음식 필요성 유튜브 장장 2개월 식이요법 ...,가수 전효성이 다이어트 영상을 공개해 화제다.\n\n\n\n전효성이 30일 오후 자...,0


In [164]:
NUM_LABELS = max(set(df['label'])) + 1
NUM_LABELS

58

In [165]:
text_to_label = dict(labels_csv)

In [166]:
label_to_text = {}
for label in text_to_label:
    label_to_text[int(text_to_label[label])] = label

In [167]:
label_to_text

{0: '문화>방송_연예',
 1: '국제>중국',
 2: '사회>사회일반',
 3: '정치>국회_정당',
 4: '사회>교육_시험',
 5: 'IT_과학>모바일',
 6: '정치>행정_자치',
 7: '사회>사건_사고',
 8: '문화>문화일반',
 9: '정치>청와대',
 10: '사회>여성',
 11: '국제>유럽_EU',
 12: '경제>자원',
 13: '국제>미국_북미',
 14: '사회>날씨',
 15: '사회>의료_건강',
 16: 'IT_과학>콘텐츠',
 17: '경제>증권_증시',
 18: '정치>선거',
 19: '국제>일본',
 20: '사회>미디어',
 21: '문화>음악',
 22: '국제>중남미',
 23: '국제>중동_아프리카',
 24: '경제>유통',
 25: '문화>출판',
 26: '경제>자동차',
 27: '사회>환경',
 28: '경제>금융_재테크',
 29: '사회>장애인',
 30: '문화>전시_공연',
 31: '경제>부동산',
 32: 'IT_과학>IT_과학일반',
 33: '문화>요리_여행',
 34: 'IT_과학>과학',
 35: '경제>서비스_쇼핑',
 36: '경제>외환',
 37: '미분류',
 38: '문화>미술_건축',
 39: '문화>생활',
 40: '경제>경제일반',
 41: '국제>아시아',
 42: '정치>정치일반',
 43: '경제>국제경제',
 44: '경제>취업_창업',
 45: 'IT_과학>보안',
 46: '정치>외교',
 47: '경제>무역',
 48: '문화>영화',
 49: '정치>북한',
 50: '경제>산업_기업',
 51: '문화>종교',
 52: '경제>반도체',
 53: '문화>학술_문화재',
 54: '사회>노동_복지',
 55: '국제>국제일반',
 56: 'IT_과학>인터넷_SNS',
 57: '국제>러시아'}

In [168]:
train, test = train, test = train_test_split(df, test_size=0.2)

print(train.shape)
print(test.shape)

(38783, 7)
(9696, 7)


In [169]:
train.head(10)

,date,title,topic,keyword,input,body,label
15030,20201126,‘2625.91’ 코스피 이틀 만에 또 최고치 기록,경제>증권_증시,"2625.91,코스피,이틀,최고,기록,코스피지수,26일,기록,사상,최고,지수,24....",성장률 코스피 최고치 코로나19 총생산 네이버 삼성에스디아이 엘지화학 셀트리온 반도...,코스피지수가 26일 또 사상 최고 기록을 세웠다. \n\n지수는 전날보다 24.37...,17
12722,20201120,이낙연 “공수처법 개정안 등 정기국회서 처리” 마지노선 제시,정치>국회_정당,"개정안,이낙연,공수처법,정기국회,처리,마지,노선,제시,국회법,절차,처리,입법과제,언...",공수처 민주당 공수처법 헌법재판소 정기국회 위원회 법사위 이낙연 의원들 본회의 개정...,"“국회법 절차대로 처리해달라”\n\n입법과제 중 가장 먼저 언급\n\n與, 내달 2...",3
1854,20201122,이랜드 그룹 랜섬웨어 공격에 도내 뉴코아 NC 등 오프라인 매장 혼란,경제>유통,"이랜드,그룹,랜섬웨어,공격,도내,뉴코아,NC,혼란,오프라인,매장,22일,이랜드,그룹...",아울렛 뉴코아 이랜드 킴스클럽 nc백화점 오프라인 동수원 nc 백화점 휴대폰 랜섬웨...,22일 새벽 이랜드 그룹 사내 네트워크가 랜섬웨어 공격을 받으면서 오프라인 매장에서...,24
7555,20201127,中 위안화고시환율 6.5755위안 0.04%↓,국제>중국,"6.5755위안,위안화,고시,환율,위안,0.04%,인민은행,중국,위안화,절상고시,인...",위안화 인민은행 중국 절상고시 고시 위안 환율 거래 달러 이날 기준 절상 대비 전장,[아시아경제 권재희 기자] 27일 중국 인민은행은 위안화를 절상고시했다. 인민은행은...,1
1117,20191128,"정치개혁 울산시민행동, 패스트트랙 처리 촉구",정치>국회_정당,"정치,개혁,울산,시민,행동,촉구,패스트,트랙,처리,정치,개혁,울산,시민,행동,보도,...",패스트트랙 본회의 울산시민행동 울산 국회의원 선거제도 공직선거법 국회 처리 개혁 권...,"정치개혁 울산시민행동은 28일 보도자료를 통해 ""정치 국회 개혁을 위한 패스트트랙을...",3
17184,20201120,"SR, 역사 안내문 응급장비 사용법 수어 안내 제공",IT_과학>모바일,"SR,역사,안내문,응급장비,사용법,수어,안내,제공,SR,대표이사,권태명,SRT,역사...",srt 청각장애인 안내문 권태명 사용법 수어통 장애인 응급장비 수서 수어안내 대표이...,[충청투데이 최정우 기자] SR(대표이사 권태명)은 SRT 역사를 이용하는 청각장애...,5
17608,20191127,"대웅제약, 역류성 식도염 신약 임상 3상 완료",IT_과학>과학,"대웅제약,3상,역류,식도염,임상,대웅제약,차세대,위식,도역,질환,펙수프라잔,임상,2...",대웅제약 펙수프라잔 위식도역류 식도염 차세대 위식 3상 블록버스터 전승호 파이프라 ...,대웅제약이 차세대 위식도역류질환 신약 ‘펙수프라잔’의 임상 3상을 완료했다고 27일...,34
18935,20201126,배달앱 10년 치킨집 사장님은 전화기를 없앴다,경제>서비스_쇼핑,"배달,치킨집,사장님,전화기,서울,동대문구,18년,치킨집,운영,매장,전화기,1대,10...",배달앱 요기요 쿠팡이츠 전화기 배달통 배민 네이버 수수료 소비자 전단지 다음달 시장...,서울 동대문구에서 18년째 치킨집을 운영하는 A(65)씨는 매장 전화기를 단 1대만...,35
1765,20201205,바레인도 화이자 코로나19 백신 긴급사용 허용,국제>중국,"바레인,코로나19,백신,긴급,사용,허용,바레인,4일,현지시간,화이자,바이오엔테크,감...",바레인 바이오엔테크 코로나19 영국 긴급사용 시노팜 코로나바이러스 뉴욕 중국 제약사...,[아시아경제 뉴욕=백종민 특파원] 바레인이 4일(현지시간) 화이자-바이오엔테크의 신...,1
18864,20201119,제14회 서덕출 문학상에 한상순 작가,문화>미술_건축,"문학상,서덕출,한상순,작가,서덕출,문학상,운영,위원회,서덕출,문학상,수상자,세상,그...",문학상 서덕출 수상자 동시인 한상순 서덕출문학상 위원장 동시집 롯데호텔 황금펜 울산...,서덕출문학상 운영위원회는 '제14회 서덕출문학상' 수상자로 '세상에서 제일 큰 키'...,38


<br>
<br>

# **Preprocessing - TRAINING SET**

In [170]:
# 문장 추출
sentences = train['input'].values
sentences[:10]

array(['성장률 코스피 최고치 코로나19 총생산 네이버 삼성에스디아이 엘지화학 셀트리온 반도체 2차 현대차 에스케이하이닉스 코스닥 비대면 삼성바이오로직스 순매도 삼성전자 대형주들 5274억 2600선 한국은행 엘지생활건강 대형주 혼조세 상승세 2575억 2516억 외국인 이틀만 코스피지수 gdp',
       '공수처 민주당 공수처법 헌법재판소 정기국회 위원회 법사위 이낙연 의원들 본회의 개정안 법무부 법제사법위원회 추미애 국회법 간담회 김태년 원내대표 박주민',
       '아울렛 뉴코아 이랜드 킴스클럽 nc백화점 오프라인 동수원 nc 백화점 휴대폰 랜섬웨어 네트워크 고객들 관계자 양효원기자킴스클럽 널점',
       '위안화 인민은행 중국 절상고시 고시 위안 환율 거래 달러 이날 기준 절상 대비 전장',
       '패스트트랙 본회의 울산시민행동 울산 국회의원 선거제도 공직선거법 국회 처리 개혁 권력 선거 행동 정치 트랙 촉구 반영 민심 신속 패스트 전횡 특혜 부패 비례 시작 보도 대표 인원 집중 연동 제도 자료 포함 시민 주장 부의 법안 강조',
       'srt 청각장애인 안내문 권태명 사용법 수어통 장애인 응급장비 수서 수어안내 대표이사 교통약자 제세동기 동탄 지제역 소화기 자동제세동기 고객준수사항 쉼터 sr 서비스 역사 이용 안내 수어 고객 마크 메뉴 렌즈 대상 장비 대표',
       '대웅제약 펙수프라잔 위식도역류 식도염 차세대 위식 3상 블록버스터 전승호 파이프라 프로톤펌프억제제 연구개발 주야간 가역 양성자펌프 중국 프로톤 적응증 가역적 경상 부작용 가시화 미국',
       '배달앱 요기요 쿠팡이츠 전화기 배달통 배민 네이버 수수료 소비자 전단지 다음달 시장점유율 결제도 정액제 치킨집 음식점 최소주문금액 1위 결제액 배달비 dh',
       '바레인 바이오엔테크 코로나19 영국 긴급사용 시노팜 코로나바이러스 뉴욕 중국 제약사 화이자 감염증 통신사 의료진 ap 백종민 ap통신 bna 백신 승인 사용 긴급 허용 관영 신종 

In [171]:
# 라벨 추출
labels = list(map(str, train['label'].values))
labels[:10]

['17', '3', '24', '1', '3', '5', '34', '35', '1', '38']

In [172]:
train_dataset = pd.DataFrame({'sentence': sentences, 'label': labels}, columns=['sentence', 'label'])

<br>
<br>

# **Preprocessing - TEST SET**

In [173]:
# 문장 추출
sentences = test['input'].values
sentences[:10]

array(['민주당 중대재해기업처벌법 산안법 형사처벌 처벌법 개정안 과징금 정책위 위원회 소상공인 3명 사업주 이낙연 정기국회 지도부 본회의 책임자 의원실',
       '임대주택 오스트리아 김현미 임대료 기숙사 월평균 만큼 000호',
       '혜민스님 혜민스 1000만 미국 종교학 서울 힐링 어린이들 직지사 해인사 뉴욕 서용건설 이관수 무소유 누리꾼 대한불교 조계종 대한불교조계종 미술품 위스타트 full 남산뷰',
       '김정은 홍콩 사업가 심재경 중국 서울 동갑내기 여배우 이젠 시청자 재경이 자작극 외도 결혼생활 수동적 장면들 전문가 미국 조력자 소드 그동안 mbn 넷플릭스 전무후무 서면 전문가적 잔혹극 달달',
       '서해 정찰기 nll 관계자 경고사격 소청 경기도 백령도 군용기 김정은 만큼 리벳 비행술 합동참모본부',
       '이근규 기념회 소시민들 충북 제천문화회관 출판기념회 제천 사람들 페이스북 에세 북콘서트 포토에세 사진들 대공연장 이야기들 안산 그동안 단양 sns 시간date 제종길 시장 현장 이야기 사람 참사 소통 고향 시민 세월 응원 관심 자리 포토 추천 열정 사진 중심 진행 참석',
       '코웨이 경쟁력 신뢰성 안전성 센터장 시험법 정선용 만큼 한국표준협회 한국 시험표준 표준원 전문성 안정성 경영성과 자원부 만족도 일상생활 고객만족도 최우선 품질경영 시험소',
       '미국 코로나19 사망자 확진자 존스홉킨스대 10만 레드필드 로버트 코비드 코로나바이러스 cnn 804명 미국구급차협회 보건복지부',
       '사업부 전문가 부행장 ai 농협은행 우리은행 국민은행 순혈주의 kb 부문장 kb국민은행 이상래 이우열 디지털금융그룹장 추진단 ict',
       '독일 중국 피셔 미국 한국 부총리 녹색당 이스라엘 남북관계 남북한 요슈카 외무장관 인천 시진핑 남북물류 인천국제해양 가능성 연립정부'],
      dtype=object)

In [174]:
# 라벨 추출
labels = list(map(str, test['label'].values))
labels[:10]

['3', '37', '51', '0', '49', '25', '50', '13', '28', '49']

In [175]:
test_dataset = pd.DataFrame({'sentence': sentences, 'label': labels}, columns=['sentence', 'label'])

In [176]:
dataset_train = train_dataset.values
dataset_test = test_dataset.values

In [177]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [178]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = []
        self.labels = []
        
        for i in dataset:
            try:
                self.sentences.append(transform([i[sent_idx]]))
                self.labels.append(np.int32(i[label_idx]))
            except Exception as e:
                print(e)
                print(len(self.sentences), len(self.labels))

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))


In [179]:
## Setting parameters
max_len = 128
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [180]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

Argument 'text' has incorrect type (expected str, got float)
8815 8815
Argument 'text' has incorrect type (expected str, got float)
10674 10674
Argument 'text' has incorrect type (expected str, got float)
24388 24388
Argument 'text' has incorrect type (expected str, got float)
28076 28076
Argument 'text' has incorrect type (expected str, got float)
35233 35233
Argument 'text' has incorrect type (expected str, got float)
4030 4030
Argument 'text' has incorrect type (expected str, got float)
9016 9016


In [181]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

In [182]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=NUM_LABELS,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [183]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [184]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [185]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [186]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [187]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [188]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [189]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(notebook.tqdm(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(notebook.tqdm(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

TypeError: ignored